In [13]:

import os
from pathlib import Path
import cv2
import numpy as np
import fitz         # PyMuPDF
from PIL import Image
from ultralytics import YOLO
from pyzbar.pyzbar import decode
import json
import validators
import requests
from urllib.parse import urlparse


In [27]:
# Paths
cwd = Path.cwd()
print("Current directory:", cwd)
ROOT_DIR = Path.cwd()
MODEL_PATH       = ROOT_DIR / "best_yolo11_merged_4datasets.pt"
PDF_PATH         = ROOT_DIR / "pdfs/АПЗ-41-чб.pdf"
OUTPUT_FOLDER    = ROOT_DIR / ""
OUTPUT_IMAGE_DIR = OUTPUT_FOLDER / "output_images"
OUTPUT_JSON_DIR  = OUTPUT_FOLDER / "output_json"
OUTPUT_ANNOT_PDF = OUTPUT_FOLDER / f"{PDF_PATH.stem}_qr_annotated.pdf"

# Create output folders
OUTPUT_IMAGE_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_JSON_DIR.mkdir(parents=True, exist_ok=True)

Current directory: /home/tox/Desktop/hackathon


In [28]:
# Load the YOLO model
model = YOLO(str(MODEL_PATH))
CLASS_NAMES = model.names
print("Loaded YOLO model with classes:", CLASS_NAMES)

Loaded YOLO model with classes: {0: 'signature', 1: 'stamp', 2: 'qr'}


In [29]:
def pdf_to_images(pdf_path: Path, zoom: float = 2.0):
    """Convert PDF pages to images with specified zoom level"""
    doc = fitz.open(str(pdf_path))
    images = []
    for page_idx in range(len(doc)):
        page = doc.load_page(page_idx)
        mat = fitz.Matrix(zoom, zoom)
        pix = page.get_pixmap(matrix=mat)
        img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n)
        if pix.n == 4:
            img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
        else:
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        images.append((page_idx, img))
    doc.close()
    return images

In [30]:
def preprocess_image(img_bgr, max_width=2000):
    """Resize image if width exceeds max_width"""
    h, w = img_bgr.shape[:2]
    if w > max_width:
        scale = max_width / w
        img_bgr = cv2.resize(img_bgr, (int(w*scale), int(h*scale)), interpolation=cv2.INTER_AREA)
    return img_bgr

In [35]:
def compute_image_hash(img):
    """Compute perceptual hash for image comparison"""
    # Resize to small size for comparison
    small = cv2.resize(img, (32, 32), interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(small, cv2.COLOR_BGR2GRAY) if len(small.shape) == 3 else small
    # Compute average hash
    avg = gray.mean()
    hash_val = (gray > avg).flatten()
    return hash_val.tobytes()

In [36]:
def images_similar(img1, img2, threshold=0.9):
    """Check if two images are similar using histogram comparison"""
    if img1.shape != img2.shape:
        img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))
    
    # Convert to grayscale
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY) if len(img1.shape) == 3 else img1
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY) if len(img2.shape) == 3 else img2
    
    # Compute histograms
    hist1 = cv2.calcHist([gray1], [0], None, [256], [0, 256])
    hist2 = cv2.calcHist([gray2], [0], None, [256], [0, 256])
    
    # Compare histograms
    correlation = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
    return correlation > threshold


In [31]:
def decode_qr_code_robust(qr_image):
    """Decode QR with preprocessing for better success rate"""
    # Try pyzbar first
    decoded_objects = decode(qr_image)
    if decoded_objects:
        return decoded_objects[0].data.decode('utf-8')
    
    # Fallback to OpenCV
    qr_detector = cv2.QRCodeDetector()
    data, bbox, _ = qr_detector.detectAndDecode(qr_image)
    if data:
        return data
    
    # Try with preprocessing
    gray = cv2.cvtColor(qr_image, cv2.COLOR_BGR2GRAY) if len(qr_image.shape) == 3 else qr_image
    preprocessed_versions = [
        gray,
        cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1],
        cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2),
    ]
    
    for processed_img in preprocessed_versions:
        decoded_objects = decode(processed_img)
        if decoded_objects:
            return decoded_objects[0].data.decode('utf-8')
    
    return None

In [38]:
def validate_qr_content(qr_data: str, timeout: int = 5):
    """Validate QR code content (URL validation)"""
    validation = {
        'data': qr_data,
        'is_url': False,
        'url_valid': False,
        'url_accessible': False,
        'is_https': False,
        'domain': None,
        'status_code': None,
        'error': None
    }
    
    if validators.url(qr_data):
        validation['is_url'] = True
        validation['url_valid'] = True
        
        parsed_url = urlparse(qr_data)
        validation['is_https'] = parsed_url.scheme == 'https'
        validation['domain'] = parsed_url.netloc
        
        try:
            response = requests.head(qr_data, timeout=timeout, allow_redirects=True)
            validation['url_accessible'] = True
            validation['status_code'] = response.status_code
            
            if response.status_code >= 400:
                validation['error'] = f"HTTP {response.status_code}"
        except requests.exceptions.RequestException as e:
            validation['url_accessible'] = False
            validation['error'] = str(e)
    
    return validation


In [46]:
# Prepare JSON structures
qr_detections = {}
qr_validation = {}
pdf_key = PDF_PATH.name

# Convert PDF to images
print("Converting PDF to images...")
pages = pdf_to_images(PDF_PATH, zoom=1)
annotated_image_paths = []

# PHASE 1: Detect all QR codes and collect their images
print("\nPhase 1: Detecting all QR codes...")
all_qr_crops = []  # Store all QR code crops
qr_metadata = []   # Store metadata for each QR

for (page_idx, img_bgr) in pages:
    img_proc = preprocess_image(img_bgr, max_width=2000)
    
    # Detect QR codes using YOLO (class 2)
    results = model.predict(source=img_proc, imgsz=1024, conf=0.25, iou=0.45, classes=[2], verbose=False)[0]
    
    if results.boxes is not None:
        boxes = results.boxes.xyxy.cpu().numpy()
        confs = results.boxes.conf.cpu().numpy()
        
        for idx, (box, conf) in enumerate(zip(boxes, confs)):
            x1, y1, x2, y2 = box.tolist()
            
            # Crop QR code region
            qr_crop = img_bgr[int(y1):int(y2), int(x1):int(x2)]
            
            all_qr_crops.append(qr_crop)
            qr_metadata.append({
                'page': page_idx,
                'index': idx,
                'bbox': (x1, y1, x2, y2),
                'confidence': float(conf),
                'original_image': img_bgr  # Keep reference for drawing later
            })

print(f"Detected {len(all_qr_crops)} QR codes total")

# PHASE 2: Group similar QR codes (find unique ones)
print("\nPhase 2: Grouping similar QR codes...")
unique_groups = []  # List of lists: each inner list contains indices of similar QRs

for i, qr_crop in enumerate(all_qr_crops):
    # Check if this QR is similar to any existing group
    found_group = False
    for group in unique_groups:
        representative_idx = group[0]
        representative_qr = all_qr_crops[representative_idx]
        
        if images_similar(qr_crop, representative_qr, threshold=0.9):
            group.append(i)
            found_group = True
            break
    
    if not found_group:
        # Create new group
        unique_groups.append([i])

print(f"Found {len(unique_groups)} unique QR code(s)")

# PHASE 3: Decode only unique QR codes
print("\nPhase 3: Decoding unique QR codes...")
decoded_results = {}  # Map group_id -> decoded_data

for group_idx, group in enumerate(unique_groups):
    representative_idx = group[0]
    representative_qr = all_qr_crops[representative_idx]
    
    print(f"  Decoding group {group_idx+1}/{len(unique_groups)} ({len(group)} instances)...")
    decoded_data = decode_qr_code_robust(representative_qr)
    
    # Store decoded data for all QRs in this group
    for qr_idx in group:
        decoded_results[qr_idx] = decoded_data

print(f"Decoding complete!")

# PHASE 4: Validate unique decoded data
print("\nPhase 4: Validating unique QR codes...")
unique_decoded_data = set(d for d in decoded_results.values() if d is not None)
validation_cache = {}  # Cache validation results

for decoded_data in unique_decoded_data:
    print(f"  Validating: {decoded_data[:50]}...")
    validation_cache[decoded_data] = validate_qr_content(decoded_data)

# PHASE 5: Build final results and annotate images
print("\nPhase 5: Building results and annotating images...")
all_qr_data = []

for (page_idx, img_bgr) in pages:
    page_key = f"page_{page_idx+1}"
    orig_h, orig_w = img_bgr.shape[:2]
    page_size = {"width": orig_w, "height": orig_h}
    
    qr_annotations = []
    
    # Find all QRs on this page
    page_qr_indices = [i for i, meta in enumerate(qr_metadata) if meta['page'] == page_idx]
    
    for qr_idx in page_qr_indices:
        meta = qr_metadata[qr_idx]
        x1, y1, x2, y2 = meta['bbox']
        width = float(x2 - x1)
        height = float(y2 - y1)
        area = width * height
        
        decoded_data = decoded_results.get(qr_idx)
        validation = validation_cache.get(decoded_data) if decoded_data else None
        
        if decoded_data:
            all_qr_data.append(decoded_data)
        
        ann_id = f"qr_{page_idx+1}_{meta['index']+1}"
        qr_annotation = {
            ann_id: {
                "category": "qr_code",
                "bbox": {"x": float(x1), "y": float(y1), "width": width, "height": height},
                "area": area,
                "confidence": meta['confidence'],
                "decoded_data": decoded_data if decoded_data else "DECODE_FAILED",
                "validation": validation
            }
        }
        qr_annotations.append(qr_annotation)
        
        # Draw on image
        color = (0, 255, 0) if decoded_data else (0, 0, 255)
        cv2.rectangle(img_bgr, (int(x1), int(y1)), (int(x2), int(y2)), color, 3)
        
        label = decoded_data[:30] + "..." if decoded_data and len(decoded_data) > 30 else (decoded_data or "FAILED")
        cv2.putText(img_bgr, label, (int(x1), int(y1)-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        if validation:
            status = "✓" if validation['url_valid'] and validation['url_accessible'] else "✗"
            cv2.putText(img_bgr, status, (int(x2)-30, int(y1)+30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)
    
    # Store page data
    qr_detections.setdefault(pdf_key, {})[page_key] = {
        "annotations": qr_annotations,
        "page_size": page_size,
        "qr_count": len(qr_annotations),
        "decoded_count": len([a for a in qr_annotations if list(a.values())[0]['decoded_data'] != "DECODE_FAILED"])
    }
    
    # Save annotated image
    out_img_path = OUTPUT_IMAGE_DIR / f"{PDF_PATH.stem}_page{page_idx+1}_qr.jpg"
    cv2.imwrite(str(out_img_path), img_bgr)
    annotated_image_paths.append(out_img_path)

print(f"Annotation complete!")


Converting PDF to images...

Phase 1: Detecting all QR codes...
Detected 21 QR codes total

Phase 2: Grouping similar QR codes...
Found 10 unique QR code(s)

Phase 3: Decoding unique QR codes...
  Decoding group 1/10 (1 instances)...
  Decoding group 2/10 (5 instances)...
  Decoding group 3/10 (6 instances)...
  Decoding group 4/10 (1 instances)...
  Decoding group 5/10 (1 instances)...
  Decoding group 6/10 (1 instances)...
  Decoding group 7/10 (1 instances)...
  Decoding group 8/10 (3 instances)...
  Decoding group 9/10 (1 instances)...
  Decoding group 10/10 (1 instances)...
Decoding complete!

Phase 4: Validating unique QR codes...
  Validating: https://www.armeta.ai/...

Phase 5: Building results and annotating images...
Annotation complete!


In [ ]:
decoded_results

In [47]:
from collections import Counter
unique_qr_codes = set(all_qr_data)
qr_frequency = Counter(all_qr_data)
most_common = qr_frequency.most_common(1)[0] if qr_frequency else (None, 0)

consistency_report = {
    'total_qr_codes': len(all_qr_data),
    'unique_qr_codes': len(unique_qr_codes),
    'is_consistent': len(unique_qr_codes) == 1,
    'most_common_qr': most_common[0],
    'most_common_frequency': most_common[1],
    'qr_frequency': dict(qr_frequency),
    'status': 'PASS' if len(unique_qr_codes) == 1 else 'FAIL',
    'optimization_stats': {
        'total_qr_detected': len(all_qr_crops),
        'unique_groups_found': len(unique_groups),
        'decoding_operations': len(unique_groups),  # Only decoded this many times!
        'time_saved_percentage': round((1 - len(unique_groups) / len(all_qr_crops)) * 100, 1) if all_qr_crops else 0
    }
}

qr_validation = {
    pdf_key: {
        "detections": qr_detections[pdf_key],
        "consistency": consistency_report
    }
}

# Save JSONs
with open(OUTPUT_JSON_DIR / "qr_detections.json", "w", encoding="utf-8") as f:
    json.dump(qr_detections, f, ensure_ascii=False, indent=2)
with open(OUTPUT_JSON_DIR / "qr_validation.json", "w", encoding="utf-8") as f:
    json.dump(qr_validation, f, ensure_ascii=False, indent=2)

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"Status: {consistency_report['status']}")
print(f"Total QR codes detected: {consistency_report['total_qr_codes']}")
print(f"Unique QR codes: {consistency_report['unique_qr_codes']}")
print(f"Most common: {consistency_report['most_common_qr']}")
print(f"Appears {consistency_report['most_common_frequency']} times")
print("\nOptimization:")
print(f"  Detected: {consistency_report['optimization_stats']['total_qr_detected']} QR codes")
print(f"  Unique groups: {consistency_report['optimization_stats']['unique_groups_found']}")
print(f"  Decoded only: {consistency_report['optimization_stats']['decoding_operations']} times")
print(f"  Time saved: ~{consistency_report['optimization_stats']['time_saved_percentage']}%")
print("="*60)

# Merge images into annotated PDF
if annotated_image_paths:
    imgs = [Image.open(str(p)) for p in annotated_image_paths]
    imgs[0].save(str(OUTPUT_ANNOT_PDF), save_all=True, append_images=imgs[1:])
    print(f"\nSaved annotated PDF: {OUTPUT_ANNOT_PDF}")
    print(f"Saved JSONs to: {OUTPUT_JSON_DIR}")


SUMMARY
Status: PASS
Total QR codes detected: 20
Unique QR codes: 1
Most common: https://www.armeta.ai/
Appears 20 times

Optimization:
  Detected: 21 QR codes
  Unique groups: 10
  Decoded only: 10 times
  Time saved: ~52.4%

Saved annotated PDF: /home/tox/Desktop/hackathon/АПЗ-41-чб_qr_annotated.pdf
Saved JSONs to: /home/tox/Desktop/hackathon/output_json
